# DVA HW4 Q2

Do not distribute or publish this code

Do not change the **#export** statements or add and other code or comments above them. They are needed for grading.

In [36]:
#export
'''
*** Imports ***
    DO NOT EDIT or add anything to this section
'''
import csv
import numpy as np  # http://www.numpy.org
import ast
from datetime import datetime
from math import log, floor, ceil
import random

## Update 'Utility' class

Modify the Utility class's methods. You can also add additional methods as required but don't change existing methods' arguments.

In [ ]:
#export
class Utility(object):

    # This method computes entropy for information gain
    def entropy(self, class_y):
        # Input:
        #   class_y         : list of class labels (0's and 1's)

        if len(class_y) == 0:
            return 0

        p1 = class_y.count(1) / len(class_y)
        p0 = class_y.count(0) / len(class_y)
        entropy = 0

        if p1 > 0:
            entropy -= p1 * log(p1)
        if p0 > 0:
            entropy -= p0 * log(p0)

        return entropy


    def partition_classes(self, X, y, split_attribute, split_val):
        # Inputs:
        #   X               : data containing all attributes
        #   y               : labels
        #   split_attribute : column index of the attribute to split on
        #   split_val       : a numerical value to divide the split_attribute

        X_left, X_right, y_left, y_right = [], [], [], []

        for i in range(len(X)):
            if X[i][split_attribute] <= split_val:
                X_left.append(X[i])
                y_left.append(y[i])
            else:
                X_right.append(X[i])
                y_right.append(y[i])

        return (X_left, X_right, y_left, y_right)


    def information_gain(self, previous_y, current_y):
        # Inputs:
        #   previous_y: the distribution of original labels (0's and 1's)
        #   current_y:  the distribution of labels after splitting based on a particular
        #               split attribute and split value
        n_parent = len(previous_y)
        if n_parent == 0:
            return 0
        parent_entropy = self.entropy(previous_y)

        weighted_child_entropy = 0
        for child_y in current_y:
            weight = len(child_y) / n_parent
            weighted_child_entropy += weight * self.entropy(child_y)

        info_gain = parent_entropy - weighted_child_entropy

        return info_gain

    def best_split(self, X, y):
        # Inputs:
        #   X       : Data containing all attributes
        #   y       : labels
        #   TODO    : For each node find the best split criteria and return the split attribute,
        #             spliting value along with  X_left, X_right, y_left, y_right (using partition_classes)
        #             in the dictionary format {'split_attribute':split_attribute, 'split_val':split_val,
        #             'X_left':X_left, 'X_right':X_right, 'y_left':y_left, 'y_right':y_right, 'info_gain':info_gain}

        best_split = {
            'split_attribute': None,
            'split_val': None,
            'X_left': [],
            'X_right': [],
            'y_left': [],
            'y_right': [],
            'info_gain': -1
        }

        if len(X) == 0:
            return best_split

        n_features = len(X[0])

        # Try every feature and every unique value as a possible split
        for feature in range(n_features):
            values = sorted(set([row[feature] for row in X]))

            for val in values:
                X_left, X_right, y_left, y_right = self.partition_classes(X, y, feature, val)
                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                gain = self.information_gain(y, [y_left, y_right])

                if gain > best_split['info_gain']:
                    best_split = {
                        'split_attribute': feature,
                        'split_val': val,
                        'X_left': X_left,
                        'X_right': X_right,
                        'y_left': y_left,
                        'y_right': y_right,
                        'info_gain': gain
                    }

        return best_split

## Update the 'DecisionTree' and 'RandomForest' classes

Please modify the 'DecisionTree' and 'RandomForest' classes below

In [ ]:
#export
class DecisionTree(object):
    def __init__(self, max_depth):
        # Initializing the tree as an empty dictionary or list, as preferred
        self.tree = {}
        self.max_depth = max_depth
        self.util = Utility() # Instantiate Utility class

    def _create_leaf(self, y):
        """Helper function to create a leaf node (predict majority class)."""
        if not y:
            return 0
        
        ones = sum(y)
        zeros = len(y) - ones
        
        return 1 if ones > zeros else 0

    def learn(self, X, y, par_node = None, depth=0):
        # Set the root node on the first call
        if depth == 0:
            par_node = self.tree
            
        # 1. If no data/labels, create a default leaf
        if not y:
            par_node['leaf_val'] = 0 # Default to 0
            return

        # 2. If all labels are the same
        if len(set(y)) == 1:
            par_node['leaf_val'] = y[0]
            return

        # 3. If max depth is reached
        if depth == self.max_depth:
            par_node['leaf_val'] = self._create_leaf(y)
            return

        # 4. Find the best split
        split_details = self.util.best_split(X, y)

        # 5. If no good split found (no info gain or empty data)
        if not split_details or split_details['info_gain'] <= 0:
            par_node['leaf_val'] = self._create_leaf(y)
            return

        # 6. Populate the current node with split info
        par_node['split_attribute'] = split_details['split_attribute']
        par_node['split_val'] = split_details['split_val']
        
        # 7. Create left and right child nodes
        par_node['left'] = {}
        par_node['right'] = {}
        
        # 8. Recursive
        self.learn(split_details['X_left'], 
                   split_details['y_left'], 
                   par_node['left'], 
                   depth + 1)
        
        self.learn(split_details['X_right'], 
                   split_details['y_right'], 
                   par_node['right'], 
                   depth + 1)

    def classify(self, record):
        node = self.tree
        
        while True:
            if 'leaf_val' in node:
                return node['leaf_val']
            
            if 'split_attribute' not in node:
                return 0
            
            attr = node['split_attribute']
            val = node['split_val']
            
            if record[attr] <= val:
                node = node['left']
            else:
                node = node['right']
                
            if not node:
                return 0

In [39]:
#export
# This starter code does not run. You will have to add your changes and
# turn in code that runs properly.

"""
Here,
1. X is assumed to be a matrix with n rows and d columns where n is the
number of total records and d is the number of features of each record.
2. y is assumed to be a vector of labels of length n.
3. XX is similar to X, except that XX also contains the data label for each
record.
"""

"""
This skeleton is provided to help you implement the assignment.You must
implement the existing functions as necessary. You may add new functions
as long as they are called from within the given classes.

VERY IMPORTANT!
Do NOT change the signature of the given functions.
Do NOT change any part of the main function APART from the forest_size parameter.
"""


class RandomForest(object):
    num_trees = 0
    decision_trees = []

    # the bootstrapping datasets for trees
    # bootstraps_datasets is a list of lists, where each list in bootstraps_datasets is a bootstrapped dataset.
    bootstraps_datasets = []

    # the true class labels, corresponding to records in the bootstrapping datasets
    # bootstraps_labels is a list of lists, where the 'i'th list contains the labels corresponding to records in
    # the 'i'th bootstrapped dataset.
    bootstraps_labels = []

    def __init__(self, num_trees):
        # Initialization done here
        self.num_trees = num_trees
        self.decision_trees = [DecisionTree(max_depth=10) for i in range(num_trees)]
        self.bootstraps_datasets = []
        self.bootstraps_labels = []

    def _bootstrapping(self, XX, n):
        sample = [] # sampled dataset
        labels = []  # class labels for the sampled records

        total_size = len(XX)
        for _ in range(n):
            record = XX[random.randint(0, total_size - 1)]
            
            sample.append(record[:-1])
            labels.append(record[-1])

        return (sample, labels)

    def bootstrapping(self, XX):
        # Initializing the bootstap datasets for each tree
        for _ in range(self.num_trees):
            data_sample, data_label = self._bootstrapping(XX, len(XX))
            self.bootstraps_datasets.append(data_sample)
            self.bootstraps_labels.append(data_label)

    def fitting(self):
        for i in range(self.num_trees):
            X = self.bootstraps_datasets[i]
            y = self.bootstraps_labels[i]
            tree = self.decision_trees[i]
            
            tree.learn(X, y)

    def voting(self, X):
        y = []

        for record in X:
            votes = []

            for i in range(len(self.bootstraps_datasets)):
                dataset = self.bootstraps_datasets[i]

                # 1. Check if the record is OOB for this tree
                if record not in dataset:
                    # 2. Get prediction from the OOB tree
                    OOB_tree = self.decision_trees[i]
                    effective_vote = OOB_tree.classify(record)
                    votes.append(int(effective_vote))

            # 3. Tally votes
            if not votes:
                # fallback: use all trees
                fallback_votes = [int(tree.classify(record)) for tree in self.decision_trees]
                if not fallback_votes:
                    y.append(0)
                else:
                    counts = {}
                    for v in fallback_votes:
                        counts[v] = counts.get(v, 0) + 1
                    y.append(max(counts, key=counts.get))
            else:
                counts = {}
                for v in votes:
                    counts[v] = counts.get(v, 0) + 1
                y.append(max(counts, key=counts.get))

        return y

    def user(self):
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId
        """
        return 'npallapangkul3'

In [40]:
#export

# TODO: Determine the forest size according to your implementation.
# This function will be used by the autograder to set your forest size during testing
# VERY IMPORTANT: Minimum forest_size should be 10
def get_forest_size():
    forest_size = 10
    return forest_size

# TODO: Determine random seed to set for reproducibility
# This function will be used by the autograder to set the random seed to obtain the same results you achieve locally
def get_random_seed():
    random_seed = 555
    return random_seed

## Do not modify the below cell
The cell below is provided to test that your random forest classifier can be successfully built and run. Similar steps will be used to build and run your code in Gradescope. Any additional testing of functions can be done in the cells below the `run()` cell, as these will not be parsed by the autograder.

In [41]:
def run():
    np.random.seed(get_random_seed())
    # start time
    start = datetime.now()
    X = list()
    y = list()
    XX = list()  # Contains data features and data labels
    numerical_cols = set([i for i in range(0, 31)])  # indices of numeric attributes (columns)

    # Loading data set
    print("reading the data")
    with open("Wisconsin_breast_prognostic.csv") as f:
        next(f, None)
        for line in csv.reader(f, delimiter=","):
            xline = []
            for i in range(len(line)):
                if i in numerical_cols:
                    xline.append(ast.literal_eval(line[i]))
                else:
                    xline.append(line[i])

            X.append(xline[:-1])
            y.append(xline[-1])
            XX.append(xline[:])

    # Initializing a random forest.
    randomForest = RandomForest(get_forest_size())

    # printing the name
    print("__Name: " + randomForest.user()+"__")

    # Creating the bootstrapping datasets
    print("creating the bootstrap datasets")
    randomForest.bootstrapping(XX)

    # Building trees in the forest
    print("fitting the forest")
    randomForest.fitting()

    # Calculating an unbiased error estimation of the random forest
    # based on out-of-bag (OOB) error estimate.
    y_predicted = randomForest.voting(X)

    # Comparing predicted and true labels
    results = [prediction == truth for prediction, truth in zip(y_predicted, y)]

    # Accuracy
    accuracy = float(results.count(True)) / float(len(results))

    print("accuracy: %.4f" % accuracy)
    print("OOB estimate: %.4f" % (1 - accuracy))

    # end time
    print("Execution time: " + str(datetime.now() - start))

In [42]:
# Call the run() function to test your implementation
# Use this cell and any cells below for additional testing
run()

reading the data
__Name: npallapangkul3__
creating the bootstrap datasets
fitting the forest
accuracy: 0.9315
OOB estimate: 0.0685
Execution time: 0:00:16.840306
